In [75]:
from urllib.parse import urlencode

import requests

OPENALEX_HEADERS = {"User-Agent": "mailto:donny@olyneme.xyz"}
OPENALEX_API_BASE_URL = "https://api.openalex.org"

def openalex_get(path, params=None, page=1):
    url = f"{OPENALEX_API_BASE_URL}{path}"
    if params:
        url += f"?{urlencode(params)}&page={page}&per-page=200"
    return requests.get(url, headers=OPENALEX_HEADERS)

In [76]:
from toolz import get_in

from toolz import keyfilter

def pick(whitelist, d):
    return keyfilter(lambda k: k in whitelist, d)

def count_and_results(rv):
    d = rv.json()
    count = get_in(['meta', 'count'], d)
    results = [pick(['id', 'display_name', 'description'], r) for r in d['results']]
    return count, results

In [77]:
from ads_query_eval.app.bootstrap import QUERIES

qs = []
for q in QUERIES:
    q = q.replace("full:",'')
    q = q.strip('()')
    q = q.replace('_', ' ')
    qs.append(q)

qs

['"coronal mass ejection"',
 '"solar wind"',
 '"ionospheric conductivity"',
 '"space weather"',
 '"geomagnetically induced current"',
 '"solar wind" AND magnetosphere AND coupling',
 'magnetosphere AND ionosphere AND coupling',
 '"interplanetary magnetic field" AND reconnection',
 '"substorm"',
 '"particle acceleration"']

In [82]:
for q in qs:
    if ' AND ' in q:
        # TODO - set intersection of concepts
        print(q)
        for q in q.split(' AND '):
            print(q)
            rv = openalex_get("/concepts", {"search": q})
            count, results = count_and_results(rv)
            if count:
                print(count)
                print(results)
            else:
                print("no matching concepts")
        continue
    rv = openalex_get("/concepts", {"search": q})
    print(q)
    count, results = count_and_results(rv)
    if count:
        print(count)
        print(results)
    else:
        print('no matching concepts')
        rv = openalex_get("/works", {"search": q})
        count, results = count_and_results(rv)
        print(count, 'matching works')
        print(results)
    print()

"coronal mass ejection"
1
[{'id': 'https://openalex.org/C72886185', 'display_name': 'Coronal mass ejection', 'description': 'significant release of plasma and magnetic field from the solar corona'}]

"solar wind"
1
[{'id': 'https://openalex.org/C108411613', 'display_name': 'Solar wind', 'description': 'stream of charged particles released from the Sun'}]

"ionospheric conductivity"
no matching concepts
192 matching works
[{'id': 'https://openalex.org/W2029308148', 'display_name': 'On the control of magnetospheric convection by the spatial distribution of ionospheric conductivities'}, {'id': 'https://openalex.org/W2022087044', 'display_name': 'Effects of ionospheric conductivity on convective flow of plasma in the magnetosphere'}, {'id': 'https://openalex.org/W2063804354', 'display_name': 'The diurnal and latitudinal variation of auroral zone ionospheric conductivity'}, {'id': 'https://openalex.org/W2037726156', 'display_name': 'Mercury and Mars: The role of ionospheric conductivity in 